In [1]:
freq_dict = {'15-min':'15T',
            '1-hour':'1H',
            '2-hour':'2H',
            '4-hour':'4H',
            '6-hour':'6H',
            '8-hour':'8H',
            '12-hour':'12H',
            '1-day':'1D',
            '2-day':'2D',
            '7-day':'7D',
            '15-day':'15D',
            '1-month':'1M'}

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from copy import deepcopy



In [3]:
metadata_df = pd.read_csv("../metadata/metadata.csv",index_col=0)

In [4]:
city_count = {}
for i in metadata_df.index:
    city = metadata_df.ix[i]['city']
    if city in city_count.keys():
        city_count[city] += 1
    else:
        city_count[city] = 1

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/ipykernel/__main__.py:3: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  app.launch_new_instance()


In [5]:
city = 'Boulder'

In [6]:
st = pd.HDFStore(os.path.expanduser("/Users/xox/monthly-12-17.h5"), mode='r')

In [7]:
freq='1M'

In [8]:
city_data = metadata_df[metadata_df['city'] == city]
city_homes = city_data.index.values.astype('int')

In [9]:
len(city_homes)

58

In [10]:
homes = {}
count = 0
for freq in ['1M']:
    print(freq)
    freq_num = int(freq[:-1])
    out = []
    homes[freq] = []
    city_data = metadata_df[metadata_df['city'] == city]
    city_homes = city_data.index.values.astype('int')
    for home in city_homes:
#         print(home)
        try: 
            data_df = st['/%d' %home]
            count += 1
            homes[home] = data_df.resample(freq).mean()
        except Exception as e:
            continue


1M


In [11]:
home_ids = homes.keys()

In [12]:
home_readings = {}
for year in range(2012, 2018):
    home_readings[year] = {}
for hid in home_ids:
    for year in range(2012, 2018):
        try:
            year_readings = homes[hid][str(year)]
            if len(year_readings) < 12:
                continue
            for m in range(12):
                old_index = year_readings.index[m]
                new_index = str(old_index)[:7]
                year_readings = year_readings.rename(index={old_index:new_index})
                home_readings[year][hid] = year_readings
                
        except:
            continue

In [13]:
# with all observed aggregate readings
all_aggregate = {}
for year in range(2013, 2018):
    all_aggregate[year] = {}
    for hid in home_readings[year].keys():
        try:
            if home_readings[year][hid]['use'].isnull().sum() == 0:
                all_aggregate[year][hid] = home_readings[year][hid]
        except Exception as e:
            continue
for year in range(2013, 2018):
    print(year, len(all_aggregate[year].keys()))

2013 0
2014 0
2015 32
2016 23
2017 16


In [16]:
year = 2015

In [19]:
# app_nan = all_aggregate[year][9121].isnull().sum()
# print(app_nan)

In [20]:
app_count = {}
for year in range(2013, 2018):
    app_count[year] = {}
    for hid in all_aggregate[year].keys():
        app_nan = all_aggregate[year][hid].isnull().sum()
        for app in app_nan.keys():
            if app_nan[app] == 0:
                if app in app_count[year]:
                    app_count[year][app] += 1
                else:
                    app_count[year][app] = 1

In [21]:
df = pd.DataFrame(app_count)

In [26]:
app_count

{2013: {},
 2014: {},
 2015: {'air1': 28,
  'air2': 4,
  'bathroom1': 8,
  'bedroom1': 4,
  'car1': 1,
  'clotheswasher1': 22,
  'clotheswasher_dryg1': 1,
  'diningroom1': 1,
  'dishwasher1': 26,
  'disposal1': 3,
  'drye1': 22,
  'dryg1': 2,
  'furnace1': 27,
  'furnace2': 2,
  'garage1': 2,
  'garage2': 1,
  'heater1': 3,
  'jacuzzi1': 7,
  'kitchen1': 2,
  'kitchenapp1': 17,
  'kitchenapp2': 9,
  'lights_plugs1': 12,
  'lights_plugs2': 5,
  'lights_plugs3': 3,
  'lights_plugs4': 1,
  'lights_plugs5': 1,
  'lights_plugs6': 1,
  'livingroom1': 4,
  'livingroom2': 1,
  'microwave1': 10,
  'office1': 5,
  'outsidelights_plugs1': 1,
  'oven1': 12,
  'range1': 8,
  'refrigerator1': 23,
  'refrigerator2': 4,
  'use': 32,
  'venthood1': 1,
  'waterheater1': 1},
 2016: {'air1': 19,
  'air2': 3,
  'bathroom1': 5,
  'bedroom1': 2,
  'car1': 1,
  'clotheswasher1': 16,
  'diningroom1': 1,
  'dishwasher1': 18,
  'disposal1': 2,
  'drye1': 17,
  'dryg1': 1,
  'furnace1': 19,
  'furnace2': 2,
  'ga

In [22]:
df[(df > 100).any(1)]

,2013,2014,2015,2016,2017


In [23]:
candidate_app = df[(df > 30).all(1)].index.tolist()

In [24]:
candidate_app = ['air1', 'clotheswasher1', 'dishwasher1', 'drye1', 'furnace1', 'kitchenapp1', 'microwave1', 'refrigerator1', 'use']

In [25]:
c = 0
overall = 0
cand = {}
for year in range(2013, 2018):
    cand[year] = {}
    for hid in all_aggregate[year].keys():
        if all_aggregate[year][hid].isnull().sum()[candidate_app].sum() == 0:
#             print(year, hid)
            df = pd.DataFrame(all_aggregate[year][hid])
            candidate = df.iloc[:, df.columns.isin(candidate_app)]
            if len(candidate.columns) == len(candidate_app):
                cand[year][hid] = candidate
    print(year, len(cand[year].keys()))
    overall += len(cand[year].keys())
print('overall', overall)

2013 0
2014 0
2015 0
2016 0
2017 0
overall 0


/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/pandas/core/series.py:705: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self.loc[key]


In [492]:
def get_dataset(selected_years, selected_appliance, all_observed, num_columns=1):
    data = {}
    for year in selected_years:
        data[year] = {}
        for hid in all_aggregate[year].keys():
            df = pd.DataFrame(all_aggregate[year][hid])
            candidate = df.iloc[:, df.columns.isin(selected_appliance)]
            if all_observed:
                if len(candidate.columns) == len(selected_appliance) and candidate.isnull().sum().sum() == 0:
                    data[year][hid] = candidate
            else:
                if len(candidate.columns) >= num_columns:
                    data[year][hid] = candidate

    return data
            

In [493]:
def get_info(data):
    overall = 0
    for year in data.keys():
        print(year, len(data[year].keys()))
        overall += len(data[year].keys())
    print("overall: ", overall)

In [504]:
selected_years = [2013, 2014, 2015, 2016, 2017]
selected_appliance = ['air1', 'clotheswasher1', 'dishwasher1', 'furnace1',  'microwave1', 'refrigerator1', 'use']
all_observed = True

In [505]:
data = get_dataset(selected_years, selected_appliance, all_observed, 2)
get_info(data)

2016 85
2017 53
2013 21
2014 59
2015 101
overall:  319


In [503]:
np.save("./data-2013-2017-observed.npy", data)